In [1]:
import nibabel as nib
import numpy as np
import torchio as tio
import os

C:\Users\wijflo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tuple_product(*args):
    product = 1
    for element in args:
        product *= element
    return product

In [10]:
def resize_images_to_reference (source_path, destination_path, ref_image_path=None, modification_string="", inclusion_string="", save_ref=False):
    files = os.listdir(source_path)

    # Select files to process.
    nifti_files = [file for file in files if (file.endswith('.nii.gz')) & (inclusion_string in file)]
    nii_img_shapes = []

    # First get a list of image shapes.
    for file in nifti_files:
        file_path = os.path.join(source_path, file)
        nii_img = nib.load(file_path)
        nii_img_shapes.append(nii_img.shape)
    
    nifti_files_to_process = nifti_files.copy()

    # If no reference image is used, dynamically choose a reference among selected files by choosing the biggest image.
    if ref_image_path is None:
        index_of_max_product = max(range(len(nii_img_shapes)), key=lambda i: tuple_product(nii_img_shapes[i]))
        del nifti_files_to_process[index_of_max_product]
        
        reference_filename = nifti_files[index_of_max_product]
        reference_image = tio.ScalarImage(os.path.join(source_path, reference_filename))
        
    else:
        reference_filename = os.path.basename(ref_image_path)
        reference_image = tio.ScalarImage(ref_image_path)
    
    # Define the reference image name, adding the modification string if necessary.
    split_name = reference_filename.split(".")
    if modification_string != "":
        split_name[0] = split_name[0] + modification_string
    new_reference_img_name = ".".join(split_name)

    if save_ref:
        reference_image.save(os.path.join(destination_path, new_reference_img_name))

    print("Processed reference image ", reference_filename)

    # Use tio.Resample to resize all images to match the shape of the reference image.
    for file in nifti_files_to_process:
        split_name = file.split("_")
        file_path = os.path.join(source_path, file)
        
        image_to_resize = tio.ScalarImage(file_path)
        normalized_image = tio.Resample(target=reference_image)(image_to_resize)
        
        split_name = file.split(".")
        if modification_string != "":
            split_name[0] = split_name[0] + modification_string
        new_img_name = ".".join(split_name)

        normalized_image.save(os.path.join(destination_path, new_img_name))
        
        print("Processed image ", file)

In [17]:
basedir = "D:\\ADS_Inference\\Original_ADS"
folder_list = [folder for folder in os.listdir(basedir) if (".ipynb" not in folder) ]

for folder in folder_list:
    folder_path = os.path.join(basedir, folder)
    resize_images_to_reference(folder_path, folder_path, "Normalized")

Processed reference image  2018-104_01-10087-D0MR_b0.nii.gz
Processed image  2018-104_01-10087-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10113-D0MR_b0.nii.gz
Processed image  2018-104_01-10113-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10114-D0MR_b0.nii.gz
Processed image  2018-104_01-10114-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10116-D0MR_b0.nii.gz
Processed image  2018-104_01-10116-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10117-D0MR_b0.nii.gz
Processed image  2018-104_01-10117-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10118-D0MR_b0.nii.gz
Processed image  2018-104_01-10118-D0MR_DWI.nii.gz


In [11]:
resize_images_to_reference("D:\\data_ETIS_781\\TOF3D", "D:\\data_ETIS_781\\Resized_Images\\TOF3D", ref_image_path="D:\\data_ETIS_781\\Resized_Images\\SWI\\2018-104_09-10556-D0MR_401_SWIp_2_Resized_MIN.nii.gz", modification_string="Resized")

Processed reference image  2018-104_09-10556-D0MR_401_SWIp_2_Resized_MIN.nii.gz
Processed image  2018-104_01-10087-D0MR_8_TOF_3D_NEW_wo_ARTEFACTS_fl3d1r_t50.nii.gz
Processed image  2018-104_01-10113-D0MR_6_3D_TOF_LARGE_.nii.gz
Processed image  2018-104_01-10114-D0MR_6_3D_TOF_LARGE_.nii.gz
Processed image  2018-104_01-10116-D0MR_8_TOF_3D_WILLIS_FIN.nii.gz
Processed image  2018-104_01-10117-D0MR_6_3D_TOF_LARGE.nii.gz
Processed image  2018-104_01-10118-D0MR_8_TOF_3D_RAPIDE.nii.gz
Processed image  2018-104_01-10120-D0MR_501_3D_TOF_HS.nii.gz
Processed image  2018-104_01-10124-D0MR_5_3D_TOF_HS.nii.gz
Processed image  2018-104_01-10126-D0MR_8_TOF_3D.nii.gz
Processed image  2018-104_01-10128-D0MR_5_3D_TOF_HS.nii.gz
Processed image  2018-104_01-10129-D0MR_7_TOF_3D_NEW_wo_ARTEFACTS.nii.gz
Processed image  2018-104_01-10136-D0MR_7_TOF_3D_NEW_wo_ARTEFACTS.nii.gz
Processed image  2018-104_01-10151-D0MR_6_3D_TOF_LARGE.nii.gz
Processed image  2018-104_01-10165-D0MR_7_TOF_50_3D_40_A_70_ANS.nii.gz
Proc

In [8]:
resize_images_to_reference("D:\\data_ETIS_781\\SWI", "D:\\data_ETIS_781\\Resized_Images\\SWI", modification_string="Resized", save_ref=True)

Processed reference image  2018-104_09-10556-D0MR_401_SWIp_2_MIN.nii.gz
Processed image  2018-104_01-10087-D0MR_12_SWI_Images.nii.gz
Processed image  2018-104_01-10113-D0MR_9_Ax_T2_GRE.nii.gz
Processed image  2018-104_01-10114-D0MR_7_Ax_3D_SWAN+CARTO.nii.gz
Processed image  2018-104_01-10116-D0MR_7_AXIAL_T2_EG.nii.gz
Processed image  2018-104_01-10117-D0MR_7_Ax_T2_GRE.nii.gz
Processed image  2018-104_01-10118-D0MR_12_SWI_Images.nii.gz
Processed image  2018-104_01-10120-D0MR_8_Ax_3D_SWAN_FAST.nii.gz
Processed image  2018-104_01-10124-D0MR_6_Ax_3D_SWAN_FAST.nii.gz
Processed image  2018-104_01-10126-D0MR_12_T2_AX_EG_QUIET.nii.gz
Processed image  2018-104_01-10128-D0MR_7_Ax_3D_SWAN_FAST.nii.gz
Processed image  2018-104_01-10129-D0MR_11_SWI_Images.nii.gz
Processed image  2018-104_01-10136-D0MR_11_SWI_Images.nii.gz
Processed image  2018-104_01-10151-D0MR_8_Ax_T2_GRE.nii.gz
Processed image  2018-104_01-10165-D0MR_13_T2_EG_AX.nii.gz
Processed image  2018-104_01-10167-D0MR_9_AX_T2_EG_HEMO.nii.g

In [12]:
resize_images_to_reference("D:\\data_ETIS_781\\MASK", "D:\\data_ETIS_781\\Resized_Images\\MASK", ref_image_path="D:\\data_ETIS_781\\Resized_Images\\SWI\\2018-104_09-10556-D0MR_401_SWIp_2_Resized_MIN.nii.gz", modification_string="Resized")

Processed reference image  2018-104_09-10556-D0MR_401_SWIp_2_Resized_MIN.nii.gz
Processed image  2018-104_01-10087-D0MR_S12_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10113-D0MR_S9_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10114-D0MR_S7_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10116-D0MR_S7_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10117-D0MR_S7_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10118-D0MR_S12_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10120-D0MR_S8_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10124-D0MR_S6_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10126-D0MR_S12_202312051300_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10128-

In [7]:
source_basedir = "D:\\data_processed_ETIS\\Intensity_Normalized_Images\\SkullStripped_Zscore_Normalize"
source_folder_list = [folder for folder in os.listdir(source_basedir)]
destination_basedir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training"
destination_folder_list = [folder for folder in os.listdir(destination_basedir)]

for source_folder, destination_folder in zip(source_folder_list, destination_folder_list):
    source_folder_path = os.path.join(source_basedir, source_folder)
    destination_folder_path = os.path.join(destination_basedir, destination_folder)
    resize_images_to_reference(source_folder_path, destination_folder_path, ref_image_path="D:\\pythonscripts_ETIS\\Reference_Image\\DAGMNet_Training\\Reference_DWI_ADS_DAGMNet_Training.nii.gz", modification_string="Training")

Processed reference image  Reference_DWI_ADS_DAGMNet_Training.nii.gz
Processed image  2018-104_01-10087-D0MR_12_SWI_Images_swi3d1r_SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10113-D0MR_9_Ax_T2_GRE__SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10114-D0MR_7_Ax_3D_SWAN+CARTO__SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10116-D0MR_7_AXIAL_T2_EG_fl2d1_SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10117-D0MR_7_Ax_T2_GRE__SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10118-D0MR_12_SWI_Images_swi3d1r_SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10120-D0MR_8_Ax_3D_SWAN_FAST__SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10124-D0MR_6_Ax_3D_SWAN_FAST__SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10126-D0MR_12_T2_AX_EG_QUIET_qfl2d1r_SkullStripped_Normalized.nii.gz
Processed image  2018-104_01-10128-D0MR_7_Ax_3D_SWAN_FAST__SkullStripped_Normalized.nii.gz
Processed image  2

In [23]:
ref_img_raw = tio.ScalarImage(("D:\\ADS_Algorithm\\ADSv1.3\\data\\examples\\Subject01\\Subject01_DWI.nii.gz"))
ref_image_resized = tio.Resize((192,224,192))(ref_img_raw)
ref_image_resized.save("D:\\pythonscripts_ETIS\\Reference_Image_SkullStripping\\Reference_DWI_ADS.nii.gz")

In [2]:
ref_img_skullstripping_resized = tio.ScalarImage(("D:\\pythonscripts_ETIS\\Reference_Image\\SkullStripping\\Reference_DWI_ADS.nii.gz"))
ref_image_resized_net_train = tio.Resize((96,112,48))(ref_img_skullstripping_resized)
ref_image_resized_net_train.save("D:\\pythonscripts_ETIS\\Reference_Image\\DAGMNet_Training\\Reference_DWI_ADS_DAGMNet_Training.nii.gz")